# Tutorial 5: Work with context

In [1]:
import yaml
from pprint import pprint

from theflow import Composable, Param, Node, load


def callback(obj, type_):
    return obj.a * 2


class Sum1(Composable):
    a: int
    b: int = 10
    c: int = 10
    d: int = Param(depends_on="b", default_callback=lambda obj, type_: obj.b * 2)

    def run(self) -> int:
        return self.a + self.b + self.c


class Sum2(Composable):
    a: int

    def run(self, a, b: int, *args, **kwargs) -> int:
        return self.a + a + b


class Plus(Composable):
    a: int
    e: int
    x: Composable
    y: Composable = Node(default=Sum1, default_kwargs={"a": 100})
    m: Composable = Node(default=Sum2, default_kwargs={"a": 100})

    @Param.decorate()
    def f(self):
        return self.a + self.e

    def run(self) -> int:
        x, y, m = self.x(), self.y(), self.m(self.a, self.e)
        print(f"{x=}, {y=}, {m=}")
        return x + y + m

In [2]:
from pprint import pprint
step = Plus(a=20, e=20, x=Sum1(a=20))
step()

x=40, y=120, m=140


300

In [10]:
step2 = Plus(a=20, e=20, x=Sum1(a=20))
step2()

x=40, y=120, m=140


300

In [7]:
step.a = 1000
step()

x=40, y=120, m=1120


1280

In [9]:
step.context._msg_store

{'__global_key__': {},
 '|1693717293': {'run_id': '1693717293'},
 '|1693717293|.': {},
 '|1693717293|__progress__': {'.x': {'input': {'args': (), 'kwargs': {}},
   'output': 40,
   'status': 'run'},
  '.y': {'input': {'args': (), 'kwargs': {}}, 'output': 120, 'status': 'run'},
  '.m': {'input': {'args': (20, 20), 'kwargs': {}},
   'output': 140,
   'status': 'run'},
  '.': {'input': {'args': (), 'kwargs': {}}, 'output': 300, 'status': 'run'}},
 '|1693717293|.x': {},
 '|1693717293|.y': {},
 '|1693717293|.m': {},
 '|1693717358': {'run_id': '1693717358'},
 '|1693717358|.': {},
 '|1693717358|__progress__': {'.x': {'input': {'args': (), 'kwargs': {}},
   'output': 40,
   'status': 'run'},
  '.y': {'input': {'args': (), 'kwargs': {}}, 'output': 120, 'status': 'run'},
  '.m': {'input': {'args': (1000, 20), 'kwargs': {}},
   'output': 1120,
   'status': 'run'},
  '.': {'input': {'args': (), 'kwargs': {}}, 'output': 1280, 'status': 'run'}},
 '|1693717358|.x': {},
 '|1693717358|.y': {},
 '|16937

In [4]:
step.last_run.logs()

{'.x': {'input': {'args': (), 'kwargs': {}}, 'output': 40, 'status': 'run'},
 '.y': {'input': {'args': (), 'kwargs': {}}, 'output': 120, 'status': 'run'},
 '.m': {'input': {'args': (20, 20), 'kwargs': {}},
  'output': 140,
  'status': 'run'},
 '.': {'input': {'args': (), 'kwargs': {}}, 'output': 300, 'status': 'run'}}

In [6]:
step.context._msg_store

{'__global_key__': {},
 '|1693717293': {'run_id': '1693717293'},
 '|1693717293|.': {},
 '|1693717293|__progress__': {'.x': {'input': {'args': (), 'kwargs': {}},
   'output': 40,
   'status': 'run'},
  '.y': {'input': {'args': (), 'kwargs': {}}, 'output': 120, 'status': 'run'},
  '.m': {'input': {'args': (20, 20), 'kwargs': {}},
   'output': 140,
   'status': 'run'},
  '.': {'input': {'args': (), 'kwargs': {}}, 'output': 300, 'status': 'run'}},
 '|1693717293|.x': {},
 '|1693717293|.y': {},
 '|1693717293|.m': {}}

## View the definition of any node and param

`theflow` allows getting the definition of a node or a param with `.specs(...)`:

-------